# **Template OP on salish**

In [22]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_beaching import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching
from OP_functions_beaching import *

In [23]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [24]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [25]:
# def maskcoast():
#     maskd=np.zeros(batt.mbathy.shape)
#     for ilat in range(baty.shape[0]):
#         for jlon in range(baty.shape[1]):
#             if baty[ilat,jlon]>0:
#                 if baty[ilat+1,jlon] == 0 or baty[ilat+1,jlon+1] == 0  \
#                  or baty[ilat+1,jlon-1] == 0 or baty[ilat,jlon] == 0   \
#                  or baty[ilat,jlon+1] == 0 or baty[ilat,jlon-1] == 0   \
#                  or baty[ilat-1,jlon] == 0 or baty[ilat-1,jlon+1] == 0 or baty[ilat-1,jlon-1] == 0:
#                     maskd[0,ilat,jlon] = 1
#     return maskd

# baty = batt.mbathy[0,:,:]
# maskd=maskcoast()

# def maskcoast2(maskd):
#     maskd2=np.zeros(batt.mbathy.shape)
#     Ilat=np.where(maskd[0,:,:]==1)[0]
#     Jlon=np.where(maskd[0,:,:]==1)[1]
#     for i in range(len(np.where(maskd[0,:,:]==1)[0])):
#         ilat=Ilat[i]
#         jlon=Jlon[i]
#         maskd2[0,ilat,jlon]=1
#         #if ilat<baty.shape[0] or jlon<baty.shape[1]:
        
#     return maskd2 
# maskd2=maskcoast2(maskd)
# dist=xr.DataArray(attrs={'Distc':maskd2})
# batt['Distc']=(batt.mbathy.dims,maskd)
# batt.to_netcdf(path='/ocean/jvalenti/MOAD/grid/mesh_maskd2T201702.nc')

In [26]:
#clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
with open('clat2.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon2.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()
    
clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

## Definitions

In [27]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 15
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat))) #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [28]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 0.5)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 500) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [29]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [30]:
#Set start date time and the name of the output file
name = 'Beaching-Unbeaching' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/Beaching-Unbeaching20181001_1n_20181016_1n.nc


## Simulation

In [31]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

# filenames,variables,dimensions=filename_set(start,length,['D'],local)
# Distc = Field.from_netcdf(filenames['D'], variables['D'], dimensions,allow_time_extrapolation=True)
# field_set.add_field(Distc)

In [32]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeaching ==> /tmp/parcels-2894/libc8297f3171c38ce79338f5d35326efb4_0.so


Particle 4159 lost !! [5400.0, 0.21652662983605853, 50.435430504087485, -126.00474468878866]
Particle 4165 lost !! [5400.0, 2.8516195480083035, 50.43368866857462, -126.01979520182309]
Particle 4179 lost !! [5400.0, 4.426350744978428, 50.449033058851455, -126.06029723279072]
Particle 4180 lost !! [5400.0, 2.7238824629185574, 50.456305624467056, -126.03003669699652]
Particle 4181 lost !! [5400.0, 1.2095119702084833, 50.47720427898417, -126.00517701153844]
Particle 4188 lost !! [5400.0, 0.966754332270075, 50.46505853471198, -126.10209472818431]
Particle 4189 lost !! [5400.0, 0.16337265691026726, 50.47715287607765, -126.07699748298467]
Particle 4196 lost !! [5400.0, 1.9656032892383162, 50.47903637387752, -126.15350291324675]
Particle 4197 lost !! [5400.0, 2.63159104079387, 50.47554804940279, -126.12062493713509]
Particle 4198 lost !! [5400.0, 0.5640291533653519, 50.490084176545814, -126.09274658327396]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-GKFJSYCJ.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-GKFJSYCJ" to convert these to a NetCDF file during the run.
  0% (7200.0 of 1296000.0) |             | Elapsed Time: 0:00:00 ETA:   0:02:24

Correct cell not found for (-124.708545, 48.412230) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 393 0
            new particle indices: (yi, xi) 393 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3246082346986675e+01 7.6507800078162114e-01
Correct cell not found for (-124.814443, 48.539778) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 425 0
            new particle indices: (yi, xi) 425 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3049264933242441e+01 6.9683556969353544e-01
Particle 2499 lost !! [13770.0, 12.561517754101184, 48.413571882407744, -124.70186811802793]
Particle 2648 lost !! [12690.0, 2.8039972522967, 48.53821288209606, -124.80859332250974]


  0% (9000.0 of 1296000.0) |             | Elapsed Time: 0:00:06 ETA:   1:03:21

...

 95% (1243800.0 of 1296000.0) |######### | Elapsed Time: 0:22:17 ETA:   0:02:24

Correct cell not found for (-124.763517, 48.474914) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 409 0
            new particle indices: (yi, xi) 409 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3591741587010013e+01 6.6683724800427380e-01
Particle 1539 lost !! [1249470.0, 2.545863211507078, 48.47641659943037, -124.75673501134689]


 98% (1279800.0 of 1296000.0) |######### | Elapsed Time: 0:23:04 ETA:   0:00:33

Correct cell not found for (-124.802900, 48.524657) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 422 0
            new particle indices: (yi, xi) 421 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3270799108015382e+01 -4.4894970576149140e-03
Particle 1723 lost !! [1286010.0, 35.29714359337931, 48.52625958759627, -124.79942766121775]


100% (1296000.0 of 1296000.0) |##########| Elapsed Time: 0:23:11 Time:  0:23:11
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
